In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
from scipy import linalg

In [9]:
# Convert to torch tensors (double precision for better numerical stability)
torch.set_default_dtype(torch.double)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
####### inputs
N, k = 20, 3  # toy size
# random SPD matrices K, M
A = np.random.randn(N, N)
K = A.T @ A + np.eye(N)   # make SPD
B = np.random.randn(N, N)
M = B.T @ B + np.eye(N)   # make SPD

K = torch.from_numpy(K).to(device)
M = torch.from_numpy(M).to(device)

# reference solution
w_all, V_all = linalg.eigh(K.cpu().numpy(), M.cpu().numpy())
print("Reference eigs:", np.round(w_all[:k], 6))


X = torch.randn(N, 3, device=device)


Reference eigs: [0.038822 0.04345  0.107503]


In [11]:
# Build the neural network that maps coordinates -> k outputs per node
class MLP(nn.Module):
    def __init__(self, in_dim=3, out_dim=k, hidden=[64,64]):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last, h, dtype=torch.double))
            layers.append(nn.Tanh())
            last = h
        layers.append(nn.Linear(last, out_dim, dtype=torch.double))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)  # returns (N, k)

# Instantiate model and optimizer
model = MLP().to(device)
# initialize final layer small
for name, p in model.named_parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [12]:
max_epochs = 20_000
print_every = 1_000
loss_history = []

In [13]:
for epoch in range(1, max_epochs+1):
    optimizer.zero_grad()
    U = model(X)  # N x k

    # losses
    B = U.T @ (M @ U)        # k x k
    orth_loss = torch.norm(B - torch.eye(k, device=device))**2
    eig_loss = torch.trace(U.T @ (K @ U))

    loss = eig_loss + orth_loss
    loss.backward()
    optimizer.step()

    if epoch % print_every == 0 or epoch == 1:
        approx_vals = torch.diag(U.T @ (K @ U)).detach().cpu().numpy()
        print(f"Epoch {epoch}, total loss={loss.item():.6f}, approx eigs={np.round(approx_vals,6)}")


Epoch 1, total loss=127817.437666, approx eigs=[ 51.088554 211.408096  28.488013]
Epoch 1000, total loss=1.662506, approx eigs=[0.275045 0.529916 0.371402]
Epoch 2000, total loss=1.440070, approx eigs=[0.349602 0.323025 0.364361]
Epoch 3000, total loss=1.286200, approx eigs=[0.381656 0.160491 0.398815]
Epoch 4000, total loss=1.204613, approx eigs=[0.374709 0.129392 0.39727 ]
Epoch 5000, total loss=1.125359, approx eigs=[0.365145 0.114234 0.400342]
Epoch 6000, total loss=1.037831, approx eigs=[0.348379 0.108926 0.390713]
Epoch 7000, total loss=0.908253, approx eigs=[0.327004 0.111241 0.335865]
Epoch 8000, total loss=0.795810, approx eigs=[0.30635  0.112973 0.278949]
Epoch 9000, total loss=0.680712, approx eigs=[0.283244 0.105867 0.21661 ]
Epoch 10000, total loss=0.552890, approx eigs=[0.246269 0.089027 0.168017]
Epoch 11000, total loss=0.466922, approx eigs=[0.204893 0.077492 0.149893]
Epoch 12000, total loss=0.397425, approx eigs=[0.161452 0.075962 0.134588]
Epoch 13000, total loss=0.2

In [14]:
# ==== Final result ====
with torch.no_grad():
    U_final = model(X)
    UKU = U_final.T @ (K @ U_final)
    mu, _ = torch.linalg.eigh(UKU)
    print("\nLearned Ritz values:", np.round(mu.cpu().numpy(), 6))
    print("Reference eigs:     ", np.round(w_all[:k], 6))


Learned Ritz values: [0.042169 0.049199 0.121183]
Reference eigs:      [0.038822 0.04345  0.107503]
